In [ ]:
import pandas as pd
from lp_crawler import LP

In [ ]:
lp = LP("아이디", "비밀번호")
lp.login()

In [ ]:
date ="2022.12.14"

url = "{}/S22/S22A10/admin/support/ajax/getLectureroomCalendarList.do".format(lp.host)
form_data = {
    "searchEduArea": "0001",
    "searchEduPlaceCode": "P1",
    "startDt": date,
    "endDt": date,
    "searchStartDt": date,
    "searchEndDt": date,
    "searchType": "REG_USER_NAME"
}
res = lp.lp_session.post(url, data=form_data)
df_src = pd.DataFrame(res.json()["lectureroomCalendarList"])

kv_dict = {
    "deptName" : "예약자 부서",
    "eduPlaceName" : "건물명",
    "lectureroomNm" : "강의실명",
    "ho" : "호실",
    "userName" : "예약자",
    "eduName" : "교육명",
    "startTime" : "시작시간",
    "endTime" : "종료시간",
    "eduDts" : "일자"
}
df_src = df_src[kv_dict.keys()]
df_src.rename(columns=kv_dict, inplace=True)

new_result = []
for item in df_src.iterrows():
    for idx, _ in enumerate(item[1]["예약자"].split("|")):
        new_result.append({
            "예약자 부서" : item[1]["예약자 부서"].split("|")[idx],
            "건물명" : item[1]["건물명"],
            "강의실명" : item[1]["강의실명"],
            "호실" : item[1]["호실"],
            "예약자" : item[1]["예약자"].split("|")[idx],
            "교육명" : item[1]["교육명"].split(",")[idx],
            "시작시간" : item[1]["시작시간"].split(",")[idx],
            "종료시간" : item[1]["종료시간"].split(",")[idx],
            "일자" : date.replace(".", "")
        })
df = pd.DataFrame(new_result)
df = df[(df["예약자"] != "")]
df.reset_index(drop=True, inplace=True)
df